### Task: Text - to - Image using Image Captioning methods -- {BLIP, ClipCap, GIT, Vit-Gpt2} 

### Imports

In [ ]:
import pickle
import matplotlib.pyplot as plt
from PIL import Image
import json
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from trectools import TrecQrel, TrecRun, TrecEval
from nltk.translate.bleu_score import sentence_bleu
from rank_bm25 import BM25Okapi

### Original Flickr8k Dataset

In [ ]:
print("Original captions - Queries")
print()
with open("flickr8k_captionid.pkl", "rb") as f:
    flickr8k_captionid = pickle.load(f)
print("Caption IDs: ", len(flickr8k_captionid), flickr8k_captionid[:2])
print()
with open("flickr8k_caption_text.pkl", "rb") as f:
    flickr8k_caption_text = pickle.load(f)
print("Caption Text: ",len(flickr8k_caption_text), flickr8k_caption_text[:2])
print()
print("Original Images - Retrieval set")
print()
with open("flickr8k_imagefile.pkl", "rb") as f:
    flickr8k_imagefile = pickle.load(f)
print("Image IDs: ",len(flickr8k_imagefile), flickr8k_imagefile[:2])
print()

src = "./original_images/"
name = src + flickr8k_imagefile[0]
im = Image.open(name)
plt.imshow(im)
plt.show()

name = src + flickr8k_imagefile[1]
im = Image.open(name)
plt.imshow(im)
plt.show()

In [ ]:
# DICTIONARY

# import pandas as pd
# df = pd.read_csv("flickr8k_expert_annotations.csv")
# print("Unique Image ids", len(set(df['iid'])))
# print("Unique Caption ids", len(set(df['cid'])))
# df
# dictionary={}
# for i in range(len(df)):
#     if str(df.iloc[i]['cid']) in flickr8k_captionid:
#         if str(df.iloc[i]['cid']) not in list(dictionary.keys()):
#             d = {}
#             d[df.iloc[i]['iid']] = df.iloc[i]['annotation']
#             dictionary[str(df.iloc[i]['cid'])] = d
#         else:
#             d_new = dictionary[str(df.iloc[i]['cid'])]
#             d_new[df.iloc[i]['iid']] = df.iloc[i]['annotation']
#             dictionary[str(df.iloc[i]['cid'])] = d_new
    
# print(len(dictionary))
# dictionary['2549968784_39bfbe44f9.jpg#2']
# a 1 indicating that the caption does not describe the image at all, 
# a 2 indicating the caption describes minor aspects of the image but does not describe the image, 
# a 3 indicating that the caption almost describes the image with minor mistakes, and 
# a 4 indicating that the caption describes the image.

# path = "./dictionary_text2image.pkl"
# with open(path, 'wb') as f:
#     pickle.dump(dictionary, f)

path = "./dictionary_text2image.pkl"
with open(path, 'rb') as f:
    dictionary = pickle.load(f)
dictionary["106490881_5a2dd9b7bd.jpg#2"]

### Loading the Generated Captions

In [ ]:
import pickle
print("Generated retrieval set")

print("BLIP")
with open("generated_captions/BLIP/flickr_cap_gen_blip.pkl", "rb") as f:
    gen_blip = pickle.load(f)
print("Caption Text: ",len(gen_blip), gen_blip[0],"\n", gen_blip[1])
print()  

print("clipcap")
with open("generated_captions/clipcap/flickr_cap_gen_clipcap.pkl", "rb") as f:
    gen_clipcap = pickle.load(f)
print("Caption Text: ",len(gen_clipcap), gen_clipcap[0],"\n", gen_clipcap[1])
print() 

print("GIT")
with open("generated_captions/git/flickr_cap_gen_git.pkl", "rb") as f:
    gen_git = pickle.load(f)
print("Caption Text: ",len(gen_git), gen_git[0],"\n", gen_git[1])
print() 

print("vit_gpt2")
with open("generated_captions/vit_gpt2/flickr_cap_gen_vit_gpt2.pkl", "rb") as f:
    gen_vit_gpt2 = pickle.load(f)
print("Caption Text: ",len(gen_vit_gpt2), gen_vit_gpt2[0],"\n", gen_vit_gpt2[1])
print() 

### feature extraction

In [ ]:
# import os
# from torch import nn
# import numpy as np
# import torch
# import torch.nn.functional as nnf
# import sys
# from typing import Tuple, List, Union, Optional
# from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
# from tqdm import tqdm, trange
# import PIL.Image
# from IPython.display import Image 
# import pickle
# from transformers import AutoTokenizer, AutoModel
# import torch
# import torch.nn.functional as F


# def mean_pooling(model_output, attention_mask):
#     token_embeddings = model_output[0] #First element of model_output contains all token embeddings
#     input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
#     return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-distilroberta-v1')
# model = AutoModel.from_pretrained('sentence-transformers/all-distilroberta-v1')

# # Tokenize sentences
# encoded_input = tokenizer(flickr8k_caption_text, padding=True, truncation=True, return_tensors='pt')

# # Compute token embeddings
# with torch.no_grad():
#     model_output = model(**encoded_input)

# # Perform pooling
# sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# # Normalize embeddings
# sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

# np.save("flickr8k_caption_original_queries_sbert.npy",sentence_embeddings)

In [ ]:
# # Tokenize sentences
# # encoded_input = tokenizer(gen_blip, padding=True, truncation=True, return_tensors='pt') #blip
# # encoded_input = tokenizer(gen_clipcap, padding=True, truncation=True, return_tensors='pt') #clipcap
# # encoded_input = tokenizer(gen_git, padding=True, truncation=True, return_tensors='pt') #git
# encoded_input = tokenizer(gen_vit_gpt2, padding=True, truncation=True, return_tensors='pt') #vit-gpt2

# # Compute token embeddings
# with torch.no_grad():
#     model_output = model(**encoded_input)

# # Perform pooling
# sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# # Normalize embeddings
# sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

# # np.save("gen_blip.npy",sentence_embeddings)
# # np.save("gen_clipcap.npy",sentence_embeddings)
# # np.save("gen_git.npy",sentence_embeddings)
# np.save("gen_vit_gpt2.npy",sentence_embeddings)

## Okapi BM25

In [ ]:
# corpus = [
#     "Hello there good man!",
#     "It is quite windy in London",
#     "How is the weather today?"
# ]

# tokenized_corpus = [doc.split(" ") for doc in corpus]

# bm25 = BM25Okapi(tokenized_corpus)
# query = "windy London"
# tokenized_query = query.split(" ")

# doc_scores = bm25.get_scores(tokenized_query)
# print(doc_scores)
# bm25.get_top_n(tokenized_query, corpus, n=1)

In [ ]:
corpus = gen_blip
tokenized_corpus = [doc.split(' ') for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

for i in range(len(flickr8k_caption_text)):
    query = str(flickr8k_caption_text[i])
    tokenized_query = query.split(" ")
    doc_scores = bm25.get_scores(tokenized_query)
    if i == 0:
        cosine_sim = doc_scores
    else:
        cosine_sim =np.vstack([cosine_sim, doc_scores])

print(cosine_sim.shape)
sorted_cosine_sim= np.flip(np.sort(cosine_sim), axis = 1)   # sorted cosin similarity values
sim_sorted_img_idx=np.flip(np.argsort(cosine_sim), axis = 1)   # positions of the sorted cosine similarity
# bm25.get_top_n(tokenized_query, corpus, n=1)

### Generated Rankings based on cosine similarity 

In [ ]:
X = np.load("text2image_features/flickr8k_caption_original_queries_sbert.npy")  # queries

Y = np.load("text2image_features/gen_blip.npy")   # generated retrieval set
# Y = np.load("text2image_features/gen_clipcap.npy")   # generated retrieval set
# Y = np.load("text2image_features/gen_git.npy")   # generated retrieval set
# Y = np.load("text2image_features/gen_vit_gpt2.npy")   # generated retrieval set

print(X.shape, Y.shape)

cosine_sim = cosine_similarity(X, Y, dense_output=True)
print(cosine_sim.shape)
sorted_cosine_sim= np.flip(np.sort(cosine_sim), axis = 1)   # sorted cosin similarity values
sim_sorted_img_idx=np.flip(np.argsort(cosine_sim), axis = 1)   # positions of the sorted cosine similarity

In [ ]:
mapped_caption_ids = np.zeros((977, 1000), dtype = np.dtype('U25'))  #mapped image ids 
for i in range(977):
    for k in range(1000):
        mapped_caption_ids[i][k] = flickr8k_imagefile[sim_sorted_img_idx[i][k]]

# np.save("./blip/sorted_cosine_sim.npy", sorted_cosine_sim)
# np.save("./blip/mapped_caption_ids.npy", mapped_caption_ids)

# np.save("./clipcap/sorted_cosine_sim.npy", sorted_cosine_sim)
# np.save("./clipcap/mapped_caption_ids.npy", mapped_caption_ids)

# np.save("./git/sorted_cosine_sim.npy", sorted_cosine_sim)
# np.save("./git/mapped_caption_ids.npy", mapped_caption_ids)

# np.save("./vit_gpt2/sorted_cosine_sim.npy", sorted_cosine_sim)
# np.save("./vit_gpt2/mapped_caption_ids.npy", mapped_caption_ids)

### Demo

In [ ]:
# Generated Dataset

with open("generated_captions/BLIP/flickr8k_test_1.json", "rb") as f:
    flickr8k_caption_text1 = json.load(f)
mapped_caption_ids1 = np.load("./blip/mapped_caption_ids.npy")

with open("generated_captions/clipcap/flickr8k_test_1.json", "rb") as f:
    flickr8k_caption_text2 = json.load(f)
mapped_caption_ids2 = np.load("./clipcap/mapped_caption_ids.npy")

with open("generated_captions/git/flickr8k_test_1.json", "rb") as f:
    flickr8k_caption_text3 = json.load(f)
mapped_caption_ids3 = np.load("./git/mapped_caption_ids.npy")

with open("generated_captions/vit_gpt2/flickr8k_test_1.json", "rb") as f:
    flickr8k_caption_text4 = json.load(f)
mapped_caption_ids4 = np.load("./vit_gpt2/mapped_caption_ids.npy")

In [ ]:
rng = 2  # queries
for j in range(1,rng):
    print("Query:", flickr8k_caption_text[j])
    src = "./original_images/"
    name = src + flickr8k_imagefile[j]
    im = Image.open(name)
    plt.imshow(im)
    plt.show()
    print("-----------------------------------")
    print("BLIP Retrieved set")
    for r in range(2):
        print(flickr8k_caption_text1[mapped_caption_ids1[j][r]])
        image = mapped_caption_ids1[j][r] 
        print()
        name = "original_images/" + image
        im = Image.open(name)
        plt.imshow(im)
        plt.show()
        
    print("-----------------------------------")
    print("CLIPCAP Retrieved set")
    for r in range(2):
        print(flickr8k_caption_text2[mapped_caption_ids2[j][r]])
        image = mapped_caption_ids2[j][r] 
        print()
        name = "original_images/" + image
        im = Image.open(name)
        plt.imshow(im)
        plt.show()
        
    print("-----------------------------------")
    print("GIT Retrieved set")
    for r in range(2):
        print(flickr8k_caption_text3[mapped_caption_ids3[j][r]])
        image = mapped_caption_ids3[j][r] 
        print()
        name = "original_images/" + image
        im = Image.open(name)
        plt.imshow(im)
        plt.show()
        
    print("-----------------------------------")
    print("VIT-GPT2 Retrieved set")
    for r in range(2):
        print(flickr8k_caption_text4[mapped_caption_ids4[j][r]])
        image = mapped_caption_ids4[j][r] 
        print()
        name = "original_images/" + image
        im = Image.open(name)
        plt.imshow(im)
        plt.show()
        
    print("##########################################################")


In [ ]:
# Demo BLIP

rng = 2  # queries
for j in range(1,rng):
    print("Query:", flickr8k_caption_text[j])
    src = "./original_images/"
    name = src + flickr8k_imagefile[j]
    im = Image.open(name)
    plt.imshow(im)
    plt.show()
    
    print("BLIP Generation Result", gen_blip[j])
    print("CLIPCAP Generation Result", gen_clipcap[j])
    print("VIT-GPT2 Generation Result", gen_vit_gpt2[j])
    print("GIT Generation Result", gen_git[j])
    print("-----------------------------------")
    

In [ ]:
# from datasets import load_metric

# bleu = load_metric("bleu")
# # CLIPCAP
# predictions1 = [["A", "young", "boy", "is", "playing", "with", "a", "frisbee", "on", "the", "beach" "."]]

# # VIT-GPT2
# predictions2 = [["a", "young" ,"boy" ,"standing" ,"on" ,"a" ,"beach" ,"holding", "a" ,"surfboard"]]

# # BLIP
# predictions3 = [["a", "young", "boy", "standing", "on", "top", "of", "a", "beach", "next", "to", "the", "ocean"]]

# # GIT
# predictions4 = [["a", "boy", "in", "blue", "shorts", "and", "blue", "shorts", "playing", "in", "the", "water","."]]

# references = [
#     [["A", "young", "boy", "in", "swimming", "trunks", "is", "walking", "with", "his", "arms", "outstretched", "on", "the", "beach"],
#     ["A", "boy", "in", "his", "blue", "swim", "shorts", "at", "the", "beach", "."],
#     ["A", "boy", "smiles", "for", "the", "camera", "at", "a", "beach", "."],
#     ["Children", "playing", "on", "the", "beach" , "."],
#     ["The", "boy", "is", "playing", "on", "the", "shore", "of", "an", "ocean" , "."]]
# ]

# print("CLIPCAP", bleu.compute(predictions=predictions1, references=references))
# print()
# print("VIT-GPT2", bleu.compute(predictions=predictions2, references=references))
# print()
# print("BLIP", bleu.compute(predictions=predictions3, references=references))
# print()
# print("GIT", bleu.compute(predictions=predictions4, references=references))
# print()

In [ ]:
# from nltk.translate.bleu_score import sentence_bleu
# reference = [
#     'A boy in his blue swim shorts at the beach .'.split(),
#     'A boy smiles for the camera at a beach .'.split(),
#     'A young boy in swimming trunks is walking with his arms outstretched on the beach .'.split(),
#     'Children playing on the beach .'.split(),
#     'The boy is playing on the shore of an ocean .'.split()
# ]
# # CLIPCAP
# predictions1 = ["A", "young", "boy", "is", "playing", "with", "a", "frisbee", "on", "the", "beach" "."]

# # VIT-GPT2
# predictions2 = ["a", "young" ,"boy" ,"standing" ,"on" ,"a" ,"beach" ,"holding", "a" ,"surfboard"]

# # BLIP
# predictions3 = ["a", "young", "boy", "standing", "on", "top", "of", "a", "beach", "next", "to", "the", "ocean"]

# # GIT
# predictions4 = ["a", "boy", "in", "blue", "shorts", "and", "blue", "shorts", "playing", "in", "the", "water","."]


# # print('Individual 1-gram: %f' % sentence_bleu(reference, candidate, weights=(1, 0, 0, 0)))
# # print('Individual 2-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 1, 0, 0)))
# # print('Individual 3-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 0, 1, 0)))
# # print('Individual 4-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 0, 0, 1)))
# print("CLIPCAP-BLEU4", sentence_bleu(reference, predictions1, weights=(0.25, 0.25, 0.25, 0.25)))
# print("VIT-GPT2-BLEU4", sentence_bleu(reference, predictions2, weights=(0.25, 0.25, 0.25, 0.25)))
# print("BLIP-BLEU4", sentence_bleu(reference, predictions3, weights=(0.25, 0.25, 0.25, 0.25)))
# print("GIT-BLEU4", sentence_bleu(reference, predictions4, weights=(0.25, 0.25, 0.25, 0.25)))

In [ ]:
# A boy in his blue swim shorts at the beach .
# A boy smiles for the camera at a beach .
# A young boy in swimming trunks is walking with his arms outstretched on the beach .
# Children playing on the beach .
# The boy is playing on the shore of an ocean .

In [ ]:
# from datasets import load_metric

# bleu = load_metric("bleu")
# # CLIPCAP
# predictions1 = [["A", "white", "dog", "laying", "on", "top", "of", "a", "wooden", "floor", "."]]

# # VIT-GPT2
# predictions2 = [["a", "white", "dog", "laying", "on", "the", "floor", "next", "to", "a", "pillow"]]

# # BLIP
# predictions3 = [["a", "large", "white", "dog", "laying", "on", "a", "tiled", "floor"]]

# # GIT
# predictions4 = [["a", "white", "dog", "laying", "on", "the", "ground", "next", "to", "a", "pool", "of", "water", "."]]

# references = [
#     [["A", "closeup", "of", "a", "white", "dog", "that", "is", "laying", "its", "head", "on", "its", "paws", "."],
#     ["A", "white", "dog", "has", "its", "head", "on", "the", "ground"],
#     ["a", "large", "white", "dog", "lying", "on", "the", "floor" , "."],
#     ["A", "white", "dog", "rests", "its", "head", "on", "the", "patio", "bricks" , "."],
#     ["A", "white", "dog", "is", "resting", "its", "head", "on", "a", "tiled", "floor", "with", "its", "eyes", "open", "."]]
# ]


# print("CLIPCAP", bleu.compute(predictions=predictions1, references=references))
# print()
# print("VIT-GPT2", bleu.compute(predictions=predictions2, references=references))
# print()
# print("BLIP", bleu.compute(predictions=predictions3, references=references))
# print()
# print("GIT", bleu.compute(predictions=predictions4, references=references))
# print()


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
reference = [
    "A closeup of a white dog that is laying its head on its paws .".split(),
    "a large white dog lying on the floor .".split(),
    "A white dog has its head on the ground .".split(),
    "A white dog is resting its head on a tiled floor with its eyes open .".split(),
    "A white dog rests its head on the patio bricks .".split()
]
# CLIPCAP
predictions1 = ["A", "white", "dog", "laying", "on", "top", "of", "a", "wooden", "floor", "."]

# VIT-GPT2
predictions2 = ["a", "white", "dog", "laying", "on", "the", "floor", "next", "to", "a", "pillow"]

# BLIP
predictions3 = ["a", "large", "white", "dog", "laying", "on", "a", "tiled", "floor"]

# GIT
predictions4 = ["a", "white", "dog", "laying", "on", "the", "ground", "next", "to", "a", "pool", "of", "water", "."]


print("CLIPCAP-BLEU4", sentence_bleu(reference, predictions1, weights=(0.25, 0.25, 0.25, 0.25)))
print("VIT-GPT2-BLEU4", sentence_bleu(reference, predictions2, weights=(0.25, 0.25, 0.25, 0.25)))
print("BLIP-BLEU4", sentence_bleu(reference, predictions3, weights=(0.25, 0.25, 0.25, 0.25)))
print("GIT-BLEU4", sentence_bleu(reference, predictions4, weights=(0.25, 0.25, 0.25, 0.25)))

In [ ]:
A closeup of a white dog that is laying its head on its paws .
a large white dog lying on the floor .
A white dog has its head on the ground .
A white dog is resting its head on a tiled floor with its eyes open .
A white dog rests its head on the patio bricks .

### Trec Formating

In [ ]:
# TREC RUNS

# sim_sorted_img_ids = np.load("./blip/mapped_caption_ids.npy")  # doc no
# sim_sorted_img_ids = np.load("./clipcap/mapped_caption_ids.npy")  # doc no
# sim_sorted_img_ids = np.load("./vit_gpt2/mapped_caption_ids.npy")  # doc no
sim_sorted_img_ids = np.load("./git/mapped_caption_ids.npy")  # doc no
print(sim_sorted_img_ids.shape)

# sorted_cosine_sim = np.load("./blip/sorted_cosine_sim.npy")  # score
# sorted_cosine_sim = np.load("./clipcap/sorted_cosine_sim.npy")  # score
# sorted_cosine_sim = np.load("./vit_gpt2/sorted_cosine_sim.npy")  # score
# sorted_cosine_sim = np.load("./git/sorted_cosine_sim.npy")  # score
# print(sorted_cosine_sim.shape)

# trec_run=[]
# for i in range(977):
#     for j in range(1000):
#         qid = i + 1
#         Q0 = "Q0"
#         docno = sim_sorted_img_ids[i][j]
#         rank = j + 1
#         score = sorted_cosine_sim[i][j]
# #         tag = "blip_4_full"
# #         tag = "clipcap_4_full"
# #         tag = "vit_gpt2_4_full"
#         tag = "git_4_full"
#         line = str(qid) + " " + Q0 + " " + str(docno) + " " + str(rank) + " " + str(score) + " " + tag
#         trec_run.append(line)
# trec_run

# # with open('trec/blip_4_full.txt', 'w') as f:
# # with open('trec/clipcap_4_full.txt', 'w') as f:
# # with open('trec/vit_gpt2_4_full.txt', 'w') as f:
# with open('trec/git_4_full.txt', 'w') as f:
#     for line in trec_run:
#         f.write(line)
#         f.write('\n')

In [ ]:
# # RANKINGS
  
# # sim_sorted_img_ids  # doc no    
# # query list = flickr8k_captionid

# trec_rel=[]
# for i in range(100):
#     d = dictionary[flickr8k_captionid[i]]
#     all_image_ids = sim_sorted_img_ids[i]
#     d_keys = list(d.keys())
#     for j in range(50):
#         qid = i + 1    
#         docno = all_image_ids[j]
#         if docno in d_keys:
#             relevance = d[docno]
#         else:
#             relevance = 0
            
#         line = str(qid) + " " + str(docno) + " " + str(relevance)
#         trec_rel.append(line)

# with open('trec/rankings_blip.txt', 'w') as f:
# with open('trec/rankings_clipcap.txt', 'w') as f:
# with open('trec/rankings_vit_gpt2.txt', 'w') as f:
# with open('trec/rankings_git.txt', 'w') as f:
#     for line in trec_rel:
#         f.write(line)
#         f.write('\n')

In [ ]:
# TrecQrel format

# sim_sorted_img_ids  # doc no    
# query list = flickr8k_captionid

# qid 0 docno relevance

# where:

# qid is the query number
# 0 is the literal 0
# docno is the id of a document in your collection
# relevance is how relevant is docno for qid
# Example:
# 1 0 aldf.1864_12_000027 1
# 1 0 aller1867_12_000032 2
# 1 0 aller1868_12_000012 0
# 1 0 aller1871_12_000640 1
# 1 0 arthr0949_12_000945 0
# 1 0 arthr0949_12_000974 1

with open("flickr8k_captionid.pkl", "rb") as f:
    flickr8k_captionid = pickle.load(f)

path = "./dictionary_text2image.pkl"
with open(path, 'rb') as f:
    dictionary = pickle.load(f)
    


trec_rel=[]
for i in range(977):
    d = dictionary[flickr8k_captionid[i]]
    all_caption_ids = sim_sorted_img_ids[i]
    d_keys = list(d.keys())
    for j in range(1000):
        qid = i + 1
        Q0 = "0"
        docno = all_caption_ids[j]
        if docno in d_keys:
            relevance = d[docno]-1
        else:
            relevance = 3
        line = str(qid) + " " + Q0 + " " + str(docno) + " " + str(relevance)
        trec_rel.append(line)

with open('trec/qrels_3_highest_text_full_3_others.txt', 'w') as f:
    for line in trec_rel:
        f.write(line)
        f.write('\n')

In [ ]:
with open("flickr8k_captionid.pkl", "rb") as f:
    flickr8k_captionid = pickle.load(f)

path = "./dictionary_text2image.pkl"
with open(path, 'rb') as f:
    dictionary = pickle.load(f)
    
# sim_sorted_img_ids -- doc no    
# query list =-- flickr8k_captionid

trec_rel=[]
for i in range(977):
    d = dictionary[flickr8k_captionid[i]]
    d_keys = list(d.keys())
    for j in range(len(d)):
        qid = i + 1
        Q0 = "0"
        docno = d_keys[j]
        score = d[docno]
        relevance = score - 1
        line = str(qid) + " " + Q0 + " " + str(docno) + " " + str(relevance)
        trec_rel.append(line)

with open('trec/qrels_3_text_condensed.txt', 'w') as f:
    for line in trec_rel:
        f.write(line)
        f.write('\n')

## RESULTS

In [ ]:
print("trec/qrels_3_highest_text_full_3_others")
qrels = TrecQrel("trec/qrels_3_highest_text_full_3_others.txt")
print(qrels.describe())

r1 = TrecRun("trec/blip_4_full.txt")
te1 = TrecEval(r1, qrels)
print("BLIP Results")
print("NDCG at 5", te1.get_ndcg(depth=5))
print("NDCG at 10", te1.get_ndcg(depth=10))
print("NDCG at 15", te1.get_ndcg(depth=15))
print("NDCG at 20", te1.get_ndcg(depth=20))
print()
print("removeUnjudged = False, binary_topical_relevance=False")
print("rbp@5: %.4f, residuals@5: %.2f, " % te1.get_rbp(p=0.5, depth= 5, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@10: %.4f, residuals@10: %.2f, " % te1.get_rbp(p=0.5, depth= 10, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@15: %.4f, residuals@15: %.2f, " % te1.get_rbp(p=0.5, depth= 15, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@20: %.4f, residuals@20: %.2f, " % te1.get_rbp(p=0.5, depth= 20, binary_topical_relevance=False, removeUnjudged = False))

print("---------------------------------")

r2 = TrecRun("trec/clipcap_4_full.txt")
te2 = TrecEval(r2, qrels)
print("CLIPCAP Results")
print("NDCG at 5", te2.get_ndcg(depth=5))
print("NDCG at 10", te2.get_ndcg(depth=10))
print("NDCG at 15", te2.get_ndcg(depth=15))
print("NDCG at 20", te2.get_ndcg(depth=20))
print()
print("removeUnjudged = False, binary_topical_relevance=False")
print("rbp@5: %.4f, residuals@5: %.2f, " % te2.get_rbp(p=0.5, depth= 5, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@10: %.4f, residuals@10: %.2f, " % te2.get_rbp(p=0.5, depth= 10, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@15: %.4f, residuals@15: %.2f, " % te2.get_rbp(p=0.5, depth= 15, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@20: %.4f, residuals@20: %.2f, " % te2.get_rbp(p=0.5, depth= 20, binary_topical_relevance=False, removeUnjudged = False))

print("---------------------------------")


r3 = TrecRun("trec/vit_gpt2_4_full.txt")
te3 = TrecEval(r3, qrels)
print("VIT-GPT2 Results")
print("NDCG at 5", te3.get_ndcg(depth=5))
print("NDCG at 10", te3.get_ndcg(depth=10))
print("NDCG at 15", te3.get_ndcg(depth=15))
print("NDCG at 20", te3.get_ndcg(depth=20))
print()
print("removeUnjudged = False, binary_topical_relevance=False")
print("rbp@5: %.4f, residuals@5: %.2f, " % te3.get_rbp(p=0.5, depth= 5, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@10: %.4f, residuals@10: %.2f, " % te3.get_rbp(p=0.5, depth= 10, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@15: %.4f, residuals@15: %.2f, " % te3.get_rbp(p=0.5, depth= 15, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@20: %.4f, residuals@20: %.2f, " % te3.get_rbp(p=0.5, depth= 20, binary_topical_relevance=False, removeUnjudged = False))

print("---------------------------------")

r4 = TrecRun("trec/git_4_full.txt")
te4 = TrecEval(r4, qrels)
print("GIT Results")
print("NDCG at 5", te4.get_ndcg(depth=5))
print("NDCG at 10", te4.get_ndcg(depth=10))
print("NDCG at 15", te4.get_ndcg(depth=15))
print("NDCG at 20", te4.get_ndcg(depth=20))
print()
print("removeUnjudged = False, binary_topical_relevance=False")
print("rbp@5: %.4f, residuals@5: %.2f, " % te4.get_rbp(p=0.5, depth= 5, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@10: %.4f, residuals@10: %.2f, " % te4.get_rbp(p=0.5, depth= 10, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@15: %.4f, residuals@15: %.2f, " % te4.get_rbp(p=0.5, depth= 15, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@20: %.4f, residuals@20: %.2f, " % te4.get_rbp(p=0.5, depth= 20, binary_topical_relevance=False, removeUnjudged = False))

In [ ]:
print("trec/qrels_3_highest_text_full_0_others")
qrels = TrecQrel("trec/qrels_3_highest_text_full_0_others.txt")
print(qrels.describe())

r1 = TrecRun("trec/blip_4_full.txt")
te1 = TrecEval(r1, qrels)
print("BLIP Results")
print("NDCG at 5", te1.get_ndcg(depth=5))
print("NDCG at 10", te1.get_ndcg(depth=10))
print("NDCG at 15", te1.get_ndcg(depth=15))
print("NDCG at 20", te1.get_ndcg(depth=20))
print()
print("removeUnjudged = False, binary_topical_relevance=False")
print("rbp@5: %.4f, residuals@5: %.2f, " % te1.get_rbp(p=0.5, depth= 5, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@10: %.4f, residuals@10: %.2f, " % te1.get_rbp(p=0.5, depth= 10, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@15: %.4f, residuals@15: %.2f, " % te1.get_rbp(p=0.5, depth= 15, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@20: %.4f, residuals@20: %.2f, " % te1.get_rbp(p=0.5, depth= 20, binary_topical_relevance=False, removeUnjudged = False))

print("---------------------------------")

r2 = TrecRun("trec/clipcap_4_full.txt")
te2 = TrecEval(r2, qrels)
print("CLIPCAP Results")
print("NDCG at 5", te2.get_ndcg(depth=5))
print("NDCG at 10", te2.get_ndcg(depth=10))
print("NDCG at 15", te2.get_ndcg(depth=15))
print("NDCG at 20", te2.get_ndcg(depth=20))
print()
print("removeUnjudged = False, binary_topical_relevance=False")
print("rbp@5: %.4f, residuals@5: %.2f, " % te2.get_rbp(p=0.5, depth= 5, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@10: %.4f, residuals@10: %.2f, " % te2.get_rbp(p=0.5, depth= 10, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@15: %.4f, residuals@15: %.2f, " % te2.get_rbp(p=0.5, depth= 15, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@20: %.4f, residuals@20: %.2f, " % te2.get_rbp(p=0.5, depth= 20, binary_topical_relevance=False, removeUnjudged = False))

print("---------------------------------")


r3 = TrecRun("trec/vit_gpt2_4_full.txt")
te3 = TrecEval(r3, qrels)
print("VIT-GPT2 Results")
print("NDCG at 5", te3.get_ndcg(depth=5))
print("NDCG at 10", te3.get_ndcg(depth=10))
print("NDCG at 15", te3.get_ndcg(depth=15))
print("NDCG at 20", te3.get_ndcg(depth=20))
print()
print("removeUnjudged = False, binary_topical_relevance=False")
print("rbp@5: %.4f, residuals@5: %.2f, " % te3.get_rbp(p=0.5, depth= 5, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@10: %.4f, residuals@10: %.2f, " % te3.get_rbp(p=0.5, depth= 10, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@15: %.4f, residuals@15: %.2f, " % te3.get_rbp(p=0.5, depth= 15, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@20: %.4f, residuals@20: %.2f, " % te3.get_rbp(p=0.5, depth= 20, binary_topical_relevance=False, removeUnjudged = False))

print("---------------------------------")

r4 = TrecRun("trec/git_4_full.txt")
te4 = TrecEval(r4, qrels)
print("GIT Results")
print("NDCG at 5", te4.get_ndcg(depth=5))
print("NDCG at 10", te4.get_ndcg(depth=10))
print("NDCG at 15", te4.get_ndcg(depth=15))
print("NDCG at 20", te4.get_ndcg(depth=20))
print()
print("removeUnjudged = False, binary_topical_relevance=False")
print("rbp@5: %.4f, residuals@5: %.2f, " % te4.get_rbp(p=0.5, depth= 5, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@10: %.4f, residuals@10: %.2f, " % te4.get_rbp(p=0.5, depth= 10, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@15: %.4f, residuals@15: %.2f, " % te4.get_rbp(p=0.5, depth= 15, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@20: %.4f, residuals@20: %.2f, " % te4.get_rbp(p=0.5, depth= 20, binary_topical_relevance=False, removeUnjudged = False))

In [ ]:
from trectools import TrecQrel, TrecRun, TrecEval

print("trec/qrels_3_text_condensed")
qrels = TrecQrel("trec/qrels_3_text_condensed.txt")
print(qrels.describe())

r1 = TrecRun("trec/blip_4_full.txt")
te1 = TrecEval(r1, qrels)
print("BLIP Results")
print("NDCG at 5", te1.get_ndcg(depth=5, removeUnjudged = True))
print("NDCG at 10", te1.get_ndcg(depth=10, removeUnjudged = True))
print("NDCG at 15", te1.get_ndcg(depth=15, removeUnjudged = True))
print("NDCG at 20", te1.get_ndcg(depth=20, removeUnjudged = True))
print()
print("removeUnjudged = True, binary_topical_relevance=False")
print("rbp@5: %.4f, residuals@5: %.2f, " % te1.get_rbp(p=0.5, depth= 5, binary_topical_relevance=False, removeUnjudged = True))
print("rbp@10: %.4f, residuals@10: %.2f, " % te1.get_rbp(p=0.5, depth= 10, binary_topical_relevance=False, removeUnjudged = True))
print("rbp@15: %.4f, residuals@15: %.2f, " % te1.get_rbp(p=0.5, depth= 15, binary_topical_relevance=False, removeUnjudged = True))
print("rbp@20: %.4f, residuals@20: %.2f, " % te1.get_rbp(p=0.5, depth= 20, binary_topical_relevance=False, removeUnjudged = True))

print("---------------------------------")

r2 = TrecRun("trec/clipcap_4_full.txt")
te2 = TrecEval(r2, qrels)
print("CLIPCAP Results")
print("NDCG at 5", te2.get_ndcg(depth=5, removeUnjudged = True))
print("NDCG at 10", te2.get_ndcg(depth=10, removeUnjudged = True))
print("NDCG at 15", te2.get_ndcg(depth=15, removeUnjudged = True))
print("NDCG at 20", te2.get_ndcg(depth=20, removeUnjudged = True))
print()
print("removeUnjudged = True, binary_topical_relevance=False")
print("rbp@5: %.4f, residuals@5: %.2f, " % te2.get_rbp(p=0.5, depth= 5, binary_topical_relevance=False, removeUnjudged = True))
print("rbp@10: %.4f, residuals@10: %.2f, " % te2.get_rbp(p=0.5, depth= 10, binary_topical_relevance=False, removeUnjudged = True))
print("rbp@15: %.4f, residuals@15: %.2f, " % te2.get_rbp(p=0.5, depth= 15, binary_topical_relevance=False, removeUnjudged = True))
print("rbp@20: %.4f, residuals@20: %.2f, " % te2.get_rbp(p=0.5, depth= 20, binary_topical_relevance=False, removeUnjudged = True))

print("---------------------------------")


r3 = TrecRun("trec/vit_gpt2_4_full.txt")
te3 = TrecEval(r3, qrels)
print("VIT-GPT2 Results")
print("NDCG at 5", te3.get_ndcg(depth=5, removeUnjudged = True))
print("NDCG at 10", te3.get_ndcg(depth=10, removeUnjudged = True))
print("NDCG at 15", te3.get_ndcg(depth=15, removeUnjudged = True))
print("NDCG at 20", te3.get_ndcg(depth=20, removeUnjudged = True))
print()
print("removeUnjudged = True, binary_topical_relevance=False")
print("rbp@5: %.4f, residuals@5: %.2f, " % te3.get_rbp(p=0.5, depth= 5, binary_topical_relevance=False, removeUnjudged = True))
print("rbp@10: %.4f, residuals@10: %.2f, " % te3.get_rbp(p=0.5, depth= 10, binary_topical_relevance=False, removeUnjudged = True))
print("rbp@15: %.4f, residuals@15: %.2f, " % te3.get_rbp(p=0.5, depth= 15, binary_topical_relevance=False, removeUnjudged = True))
print("rbp@20: %.4f, residuals@20: %.2f, " % te3.get_rbp(p=0.5, depth= 20, binary_topical_relevance=False, removeUnjudged = True))

print("---------------------------------")

r4 = TrecRun("trec/git_4_full.txt")
te4 = TrecEval(r4, qrels)
print("GIT Results")
print("NDCG at 5", te4.get_ndcg(depth=5, removeUnjudged = True))
print("NDCG at 10", te4.get_ndcg(depth=10, removeUnjudged = True))
print("NDCG at 15", te4.get_ndcg(depth=15, removeUnjudged = True))
print("NDCG at 20", te4.get_ndcg(depth=20, removeUnjudged = True))
print()
print("removeUnjudged = True, binary_topical_relevance=False")
print("rbp@5: %.4f, residuals@5: %.2f, " % te4.get_rbp(p=0.5, depth= 5, binary_topical_relevance=False, removeUnjudged = True))
print("rbp@10: %.4f, residuals@10: %.2f, " % te4.get_rbp(p=0.5, depth= 10, binary_topical_relevance=False, removeUnjudged = True))
print("rbp@15: %.4f, residuals@15: %.2f, " % te4.get_rbp(p=0.5, depth= 15, binary_topical_relevance=False, removeUnjudged = True))
print("rbp@20: %.4f, residuals@20: %.2f, " % te4.get_rbp(p=0.5, depth= 20, binary_topical_relevance=False, removeUnjudged = True))

In [ ]:
from trectools import TrecQrel, TrecRun, TrecEval

print("trec/qrels_3_text_condensed")
qrels = TrecQrel("trec/qrels_3_text_condensed.txt")
print(qrels.describe())

r1 = TrecRun("trec/blip_4_full.txt")
te1 = TrecEval(r1, qrels)
print("BLIP Results")
print("NDCG at 5", te1.get_ndcg(depth=5, removeUnjudged = True))
print("NDCG at 10", te1.get_ndcg(depth=10, removeUnjudged = True))
print("NDCG at 15", te1.get_ndcg(depth=15, removeUnjudged = True))
print("NDCG at 20", te1.get_ndcg(depth=20, removeUnjudged = True))
print()
print("removeUnjudged = True, binary_topical_relevance=False")
print("rbp@5: %.4f, residuals@5: %.2f, " % te1.get_rbp(p=0.5, depth= 5, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@10: %.4f, residuals@10: %.2f, " % te1.get_rbp(p=0.5, depth= 10, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@15: %.4f, residuals@15: %.2f, " % te1.get_rbp(p=0.5, depth= 15, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@20: %.4f, residuals@20: %.2f, " % te1.get_rbp(p=0.5, depth= 20, binary_topical_relevance=False, removeUnjudged = False))

print("---------------------------------")

r2 = TrecRun("trec/clipcap_4_full.txt")
te2 = TrecEval(r2, qrels)
print("CLIPCAP Results")
print("NDCG at 5", te2.get_ndcg(depth=5, removeUnjudged = True))
print("NDCG at 10", te2.get_ndcg(depth=10, removeUnjudged = True))
print("NDCG at 15", te2.get_ndcg(depth=15, removeUnjudged = True))
print("NDCG at 20", te2.get_ndcg(depth=20, removeUnjudged = True))
print()
print("removeUnjudged = True, binary_topical_relevance=False")
print("rbp@5: %.4f, residuals@5: %.2f, " % te2.get_rbp(p=0.5, depth= 5, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@10: %.4f, residuals@10: %.2f, " % te2.get_rbp(p=0.5, depth= 10, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@15: %.4f, residuals@15: %.2f, " % te2.get_rbp(p=0.5, depth= 15, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@20: %.4f, residuals@20: %.2f, " % te2.get_rbp(p=0.5, depth= 20, binary_topical_relevance=False, removeUnjudged = False))

print("---------------------------------")


r3 = TrecRun("trec/vit_gpt2_4_full.txt")
te3 = TrecEval(r3, qrels)
print("VIT-GPT2 Results")
print("NDCG at 5", te3.get_ndcg(depth=5, removeUnjudged = True))
print("NDCG at 10", te3.get_ndcg(depth=10, removeUnjudged = True))
print("NDCG at 15", te3.get_ndcg(depth=15, removeUnjudged = True))
print("NDCG at 20", te3.get_ndcg(depth=20, removeUnjudged = True))
print()
print("removeUnjudged = True, binary_topical_relevance=False")
print("rbp@5: %.4f, residuals@5: %.2f, " % te3.get_rbp(p=0.5, depth= 5, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@10: %.4f, residuals@10: %.2f, " % te3.get_rbp(p=0.5, depth= 10, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@15: %.4f, residuals@15: %.2f, " % te3.get_rbp(p=0.5, depth= 15, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@20: %.4f, residuals@20: %.2f, " % te3.get_rbp(p=0.5, depth= 20, binary_topical_relevance=False, removeUnjudged = False))

print("---------------------------------")

r4 = TrecRun("trec/git_4_full.txt")
te4 = TrecEval(r4, qrels)
print("GIT Results")
print("NDCG at 5", te4.get_ndcg(depth=5, removeUnjudged = True))
print("NDCG at 10", te4.get_ndcg(depth=10, removeUnjudged = True))
print("NDCG at 15", te4.get_ndcg(depth=15, removeUnjudged = True))
print("NDCG at 20", te4.get_ndcg(depth=20, removeUnjudged = True))
print()
print("removeUnjudged = True, binary_topical_relevance=False")
print("rbp@5: %.4f, residuals@5: %.2f, " % te4.get_rbp(p=0.5, depth= 5, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@10: %.4f, residuals@10: %.2f, " % te4.get_rbp(p=0.5, depth= 10, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@15: %.4f, residuals@15: %.2f, " % te4.get_rbp(p=0.5, depth= 15, binary_topical_relevance=False, removeUnjudged = False))
print("rbp@20: %.4f, residuals@20: %.2f, " % te4.get_rbp(p=0.5, depth= 20, binary_topical_relevance=False, removeUnjudged = False))

In [ ]:
from trectools import TrecQrel, TrecRun, TrecEval

# query 2

print("trec/qrels_3_text_condensed")
qrels = TrecQrel("trec/qrels_q2.txt")
# print(qrels.describe())

r1 = TrecRun("trec/blip_4_full.txt")
te1 = TrecEval(r1, qrels)
print("BLIP Results")
print(te1.get_ndcg(depth=5, removeUnjudged = True, per_query = True))

r1 = TrecRun("trec/clipcap_4_full.txt")
te1 = TrecEval(r1, qrels)
print("CLIPCAP Results")
print(te1.get_ndcg(depth=5, removeUnjudged = True, per_query = True))

r1 = TrecRun("trec/vit_gpt2_4_full.txt")
te1 = TrecEval(r1, qrels)
print("VIT-GPT2 Results")
print(te1.get_ndcg(depth=5, removeUnjudged = True, per_query = True))

r1 = TrecRun("trec/git_4_full.txt")
te1 = TrecEval(r1, qrels)
print("GIT Results")
print(te1.get_ndcg(depth=5, removeUnjudged = True, per_query = True))

In [ ]:
# CLIP Score

import torch
import torch
from PIL import Image
import torchvision.transforms as transforms

_ = torch.manual_seed(42)
from torchmetrics.multimodal import CLIPScore
metric = CLIPScore(model_name_or_path="openai/clip-vit-base-patch16")

# Read a PIL image
name = src + flickr8k_imagefile[1]
image = Image.open(name)
plt.imshow(im)
plt.show()

# Define a transform to convert PIL
# image to a Torch tensor
transform = transforms.Compose([
	transforms.PILToTensor()
])

# transform = transforms.PILToTensor()
# Convert the PIL image to Torch tensor
img_tensor = transform(image)

print(metric(img_tensor, "a young boy standing on top of a beach next to the ocean"))
print(metric(img_tensor, "A young boy is playing with a frisbee on the beach."))
print(metric(img_tensor, "a young boy standing on a beach holding a surfboard"))
print(metric(img_tensor, "a boy in blue shorts and blue shorts playing in the water."))


# BLIP Generation Result a young boy standing on top of a beach next to the ocean
# CLIPCAP Generation Result A young boy is playing with a frisbee on the beach.
# VIT-GPT2 Generation Result a young boy standing on a beach holding a surfboard
# GIT Generation Result a boy in blue shorts and blue shorts playing in the water.

In [ ]:
print(metric(img_tensor, "two boys playing near the ocean"))
print(metric(img_tensor, "a picture having two boys, one standing and another bending to touch the ocean"))
print(metric(img_tensor, "two boys playing near the ocean, one standing in blue shorts and another bending to touch the water"))
print(metric(img_tensor, "a photograph of two boys, one standing on the sea in blue shorts and another bending to touch the water"))

In [ ]:
print(metric(img_tensor, "two young boys in blue shorts playing on the ocean"))

In [ ]:
# BLIP
!python ../clipscore/clipscore.py ../clipscore/flickr8k/BLIP/flickr8k_test_clipscore.json  original_images/ --references_json ../clipscore/flickr8k/flickr8k_ref.json

In [ ]:
# CLIPCAP
!python ../clipscore/clipscore.py ../clipscore/flickr8k/clipcap/flickr8k_test_clipscore.json  original_images/ --references_json ../clipscore/flickr8k/flickr8k_ref.json

In [ ]:
# GIT
!python ../clipscore/clipscore.py ../clipscore/flickr8k/git/flickr8k_test_clipscore.json  original_images/ --references_json ../clipscore/flickr8k/flickr8k_ref.json

In [ ]:
# VIT-GPT2
!python ../clipscore/clipscore.py ../clipscore/flickr8k/vit_gpt2/flickr8k_test_clipscore.json  original_images/ --references_json ../clipscore/flickr8k/flickr8k_ref.json

In [ ]:
# # VIT-GPT2

# BLEU-1: 0.6158
# BLEU-2: 0.4201
# BLEU-3: 0.2719
# BLEU-4: 0.1721
# METEOR: 0.2010
# ROUGE: 0.4538
# CIDER: 0.4760
# SPICE: 0.1390
# CLIPScore: 0.7147
# RefCLIPScore: 0.7522

    
# # GIT
# BLEU-1: 0.6972
# BLEU-2: 0.5166
# BLEU-3: 0.3654
# BLEU-4: 0.2568
# METEOR: 0.2372
# ROUGE: 0.5097
# CIDER: 0.7464
# SPICE: 0.1767
# CLIPScore: 0.7853
# RefCLIPScore: 0.8141
    
# # CLIPCAP    
# BLEU-1: 0.6389
# BLEU-2: 0.4496
# BLEU-3: 0.2951
# BLEU-4: 0.1869
# METEOR: 0.2161
# ROUGE: 0.4752
# CIDER: 0.5466
# SPICE: 0.1498
# CLIPScore: 0.7755
# RefCLIPScore: 0.7911
    
# # BLIP

# BLEU-1: 0.7221
# BLEU-2: 0.5501
# BLEU-3: 0.4036
# BLEU-4: 0.2914
# METEOR: 0.2637
# ROUGE: 0.5411
# CIDER: 0.8993
# SPICE: 0.1974
# CLIPScore: 0.7781
# RefCLIPScore: 0.8152

# Formatting

In [ ]:
# import json

# with open("generated_captions/BLIP/flickr8k_test_1.json", "rb") as f:
#     flickr8k_caption_text1 = json.load(f)
# with open("generated_captions/clipcap/flickr8k_test_1.json", "rb") as f:
#     flickr8k_caption_text2 = json.load(f)
# with open("generated_captions/git/flickr8k_test_1.json", "rb") as f:
#     flickr8k_caption_text3 = json.load(f)
# with open("generated_captions/vit_gpt2/flickr8k_test_1.json", "rb") as f:
#     flickr8k_caption_text4 = json.load(f)

# # Generated Dataset formatting
# dict1 = {}
# d = flickr8k_caption_text4
# for i in range(len(d)):
#     d_keys = list(d.keys())
#     dict1[d_keys[i][:-4]] = d[d_keys[i]]
# dict1

# # with open("generated_captions/BLIP/flickr8k_test_clipscore.json", 'w') as f:
# # with open("generated_captions/clipcap/flickr8k_test_clipscore.json", 'w') as f:
# # with open("generated_captions/git/flickr8k_test_clipscore.json", 'w') as f:
# with open("generated_captions/vit_gpt2/flickr8k_test_clipscore.json", 'w') as f:
#     json.dump(dict1, f)

In [ ]:
# import pandas as pd
# df = pd.read_csv('annotations/captions.txt', sep=',')
# print(df.shape[0])
# df.head()

# # CLIP Score FORMAT

# dictionary={}
# for i in range(len(df)):
#     if str(df.iloc[i]['image']) in flickr8k_imagefile:
#         if str(df.iloc[i]['image'][:-4]) not in list(dictionary.keys()):
#             dictionary[str(df.iloc[i]['image'][:-4])] = [str(df.iloc[i]['caption'])]
#         else:
#             dictionary[str(df.iloc[i]['image'][:-4])].append(str(df.iloc[i]['caption']))
# dictionary

# with open("annotations/flickr8k_ref.json", "w") as f:
#     json.dump(dictionary,f)

In [ ]:
# import pandas as pd
# df = pd.read_csv('annotations/captions.txt', sep=',')
# print(df.shape[0])
# df.head()

# # COCO FORMAT

# # Create annotations
# annotations = [{"image_id" : df['image'].iloc[i], "id" : i, "caption" : df['caption'].iloc[i]} for i in range(df.shape[0])]
    
# # Create image annotations
# images = [{"id" : df['image'].iloc[i], "file_name" : df['image'].iloc[i]} for i in range(df.shape[0])]

# # Create coco_format
# coco_format = {"annotations" : annotations,
#                "images" : images,
#                "type": "captions",
#                "info": "dummy",
#                "licenses": "dummy"}
   
# with open(annotations/flickr8k_coco_format.json", "w") as f:
#     json.dump(coco_format, f, separators=(', ', ': '), ensure_ascii = False)

In [ ]:
# from pycocotools.coco import COCO
# from pycocoevalcap.eval import COCOEvalCap

# # annotation_file = "annotations/final_flickr_separateGT_test.json"  -- Results do not correspond to current coco set
# # annotation_file = "annotations/dataset_flickr8k.json"  -- not useful
# annotation_file = "annotations/flickr8k_caption_coco_format.json"
# results_file = "generated_captions/BLIP/flickr8k_test_cs.json"
# # results_file ="generated_captions/clipcap/flickr8k_test_cs.json"
# # results_file ="generated_captions/git/flickr8k_test_cs.json"
# # results_file = "generated_captions/vit_gpt2/flickr8k_test_cs.json"

# coco = COCO(annotation_file)
# coco_result = coco.loadRes(results_file)
# coco_eval = COCOEvalCap(coco, coco_result)
# coco_eval.params['image_id'] = coco_result.getImgIds()
# coco_eval.evaluate()

# for metric, score in coco_eval.eval.items():
#     print(f'{metric}: {score:.3f}')